## Installation

In [ ]:
!pip install -q "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install -q --no-deps trl peft accelerate bitsandbytes
!pip install -q xformers==0.0.22.post7

## Model Loading

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048
dtype = None # None for auto detection.
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/mistral-7b-instruct-v0.2-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = True,
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)


### Data Prep

In [4]:
xlcost_prompt = """Below is a description of a programming task. Write a response that appropriately completes the task based on the given description.

### Description:
{}

### Code:
{}"""

EOS_TOKEN = tokenizer.eos_token

def formatting_prompts_func(examples):
    descriptions = examples["text"]
    codes        = examples["code"]
    formatted_texts = []
    for description, code in zip(descriptions, codes):
        text = xlcost_prompt.format(description, code) + EOS_TOKEN
        formatted_texts.append(text)
    return { "text" : formatted_texts }

datasource = "codeparrot/xlcost-text-to-code"

from datasets import load_dataset
dataset = load_dataset("codeparrot/xlcost-text-to-code", "Python-program-level", split="train")
dataset = dataset.map(formatting_prompts_func, batched=True)


### Train the model

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

In [ ]:
trainer_stats = trainer.train()

### Inference

In [ ]:
FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    xlcost_prompt.format(
        "Continue the fibonnaci sequence.",
        "",
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

### Saving


In [ ]:
model.push_to_hub_merged("akameswa/mistral-7b-instruct-python-4bit", tokenizer, save_method = "merged_4bit_forced", token = "")